# SRU-Abfrage + Auslesen von MARC-(Unter)-Feldern

Dieses Script macht eine Katalogabfrage via SRU und liest danach vorgegebene Kontroll- und Unterelder aus dem zurückgegebenen MARCXML aus.

## Import

Benötigte Python-Bibliotheken oder einzelne Module aus Bibliotheken müssen importiert werden. Dieser modulartige Aufbau verhindert, dass Python als Grundprogram überladen, fehleranfällig und langsam wird.

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

from google.colab import files   #Diese Zeile kann mit einem # deaktiviert werden, wenn nicht auf Colab gearbeitet wird.

## SRU Grundabfrage

Im folgenden Code-Block wird die Grundabfrage der SRU festgelegt, also 
* die Basis-URL (NZ/IZ, nicht verwendete URL mit # deaktivieren. Bei der Suche in der IZ werden zusätzlich zu den Titeldatensätzen auch die Bestandesdatensätze (tag="AVA") zurückgegeben.)

und die zwingenden Parameter 
* für die Version (1.2)
* die auszuführende Operation (searchRetrieve)
* das "query="

Zusätzlich werden diese zwei optionalen Parameter festgelegt:

* MARCXML ist als Schema definiert. Dies könnte auch weggelassen (default) oder durch ein anderes Schema ersetzt werden (z.B. durch dc oder mods).
* Die maximale Zahl an Ergebnissen kann höchstens auf 100 erhöht werden. Hier werden jedoch mehr Ergebnisse ermöglicht, indem nach den ersten 50 das Startergebniss um 50 erhöht wird (also auf 51 gesetzt wird, dann auf 101 usw.).

Beispiel für eine komplette SRU Abfrage nach einer MMS-ID:

https://slsp-ube.alma.exlibrisgroup.com/view/sru/41SLSP_UBE?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=alma.mms_id=99117122326005511





In [2]:
# SRU Grundabfrage

def slsp_sru(query):
    
    # für NZ-Abfragen
    #base_url = 'https://slsp-network.alma.exlibrisgroup.com/view/sru/41SLSP_NETWORK'
    
    # für IZ-UBE-Abfragen 
    base_url = 'https://slsp-ube.alma.exlibrisgroup.com/view/sru/41SLSP_UBE'
    
    params = {'version': '1.2',
          'operation': 'searchRetrieve',
          'recordSchema' : 'marcxml',
          'maximumRecords': '50',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    
    # MARCXML der Ergebnisse anzeigen lassen durch print(xml)
    #print(xml)  
    
    records = xml.find_all('record', xmlns='http://www.loc.gov/MARC21/slim')
    num_results = len(records)
    i = 1
    while num_results == 50:
        i+=50
        params.update({'startRecord': i})
        r = requests.get(base_url, params=params)
        xml = soup(r.content)
        new_records = xml.find_all('record', xmlns='http://www.loc.gov/MARC21/slim')
        records+=new_records
        num_results = len(new_records)
        
    return records

## Index & Query

Hier wird festgelegt, in welchem Suchindex nach welchen Werten gesucht wird.

Für die Indices kann die  operation=explain verwendet werden:
* https://slsp-network.alma.exlibrisgroup.com/view/sru/41SLSP_NETWORK?version=1.2&operation=explain
* https://slsp-ube.alma.exlibrisgroup.com/view/sru/41SLSP_UBE?version=1.2&operation=explain

!Achtung! Das "alma." vor dem Index nicht vergessen! z.B. mms_id >>> **alma.mms_id**

In [3]:
# SRU Beispiel-Abfragen

#records = slsp_sru("alma.title=Bakunin")
#records = slsp_sru("alma.title=Bakunin and alma.date_of_publication=1987") 

# !Achtung! Suche nach local_field oder holding_Library funktioniert nur in der IZ!

records = slsp_sru("alma.local_field_990=bbggr")
#records = slsp_sru("alma.title=Bakunin and alma.holding_Library=112063420005511")


/usr/local/lib/python3.9/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


## Auswertung des zurückgegebenen MARCXML

Die Funktion **def parse_record_with_namespace_v2(record):** liest das XML aus.

Die Code-Blöcke für die einzelnen Kontrollfelder, resp. einzelnen Unterfelder, können kopiert und an die eigenen Bedürfnisse angepasst werden.

In [4]:
# Auswertung MARCXML

def parse_record_with_namespace_v2(record):
    
    ns = {'marc':'http://www.loc.gov/MARC21/slim'}
    xml = etree.fromstring(unicodedata.normalize('NFC', str(record)))

# Kontrollfeld ohne Unterfelder
# Für andere Kontrollfelder muss der @tag und der Variablenname (idn) angepasst werden.

    # MMSID (idn)

    idn = xml.xpath('marc:controlfield[@tag = "001"]', namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'   #Anstatt 'fail' kann man beim Nichtfinden einer MMS-Id auch '-', 'keine MMS-ID' usw. ausgeben lassen.

# Titel: 245$a (nicht wiederholbar)

    title = xml.xpath('marc:datafield[@tag = "245"]/marc:subfield[@code = "a"]', namespaces=ns)
    try:
        title = title[0].text
    except:
        title = 'no title'

# Unterfeld: gibt auch multiple Unterfelder [035$a] zurück.
# Für andere/weitere Unterfelder muss der @tag, der @code und der Variablenname (sys) angepasst werden.

    # alte Systemnummer 035$a (sys)

    sys = xml.xpath('marc:datafield[@tag = "035"]/marc:subfield[@code = "a"]', namespaces=ns)    
    if len(sys) > 0:
        sys = [element.text for element in sys]
        sys = ' \ '.join(sys)   #Trennzeichen bei mehrfachen Werten kann selber festgelegt werden (hier: \ )
    else:
        sys = 'no 035$a'   #Anstatt 'no 035$a' kann man beim Nichtfinden einer alten Systemnummer auch '-', 'fail' usw. ausgeben lassen.

# Zwei Unterfelder, nur wenn beide im Feld vorhanden sind.

    # Feld 651$a (ort) und 651$0 (gnd), nur wenn beide Unterfelder vorkommen.

    fields_651 = xml.xpath('marc:datafield[@tag = "651"]', namespaces=ns)
    
    ort = []
    gnd = []
    
    # Check the entire list of fields 651
    for field_651 in fields_651:
        
        # Check if subfield a and 0 are available
        if field_651.find('marc:subfield[@code = "0"]', namespaces=ns) is not None \
            and field_651.find('marc:subfield[@code = "a"]', namespaces=ns) is not None:
            ort.append(field_651.find('marc:subfield[@code = "a"]', namespaces=ns).text)
            gnd.append(field_651.find('marc:subfield[@code = "0"]', namespaces=ns).text)
    
    # ort is "unknown" if no value fetched
    if len(ort) == 0:
        ort = 'unknown'
    else:
        ort = '§'.join(ort)   #Trennzeichen bei mehrfachen Werten kann selber festgelegt werden (hier: §)
    
    # gnd is "unknown" if no value fetched
    if len(gnd) == 0:
        gnd = 'unknown'
    else:
        gnd = '§'.join(gnd)   #Trennzeichen bei mehrfachen Werten kann selber festgelegt werden (hier: §)

# Für das Ergebnissset müssen die auszugebenden Variablen ('idn', etc.) und die zugehörigen Spaltenüberschriften ('MMSID') definiert werden.

    meta_dict = {'MMSID':idn,
                 'Titel':title,
                 '035$a':sys,
                 'Ort':ort,
                 'GND#':gnd}
    
    return meta_dict

## Ausgabedatei

Das Ergebnissset wird in einem Dataframe (df) gespeichert und dann als Excel auf das lokale Laufwerk heruntergeladen. Dabei kann der Name der Datei (query)  angepasst werden.

In [8]:
# Output

output = [parse_record_with_namespace_v2(record) for record in records]
df = pd.DataFrame(output)

query = "sru_query_bbggr.xlsx"

df.to_excel(query, encoding="UTF-8")
files.download(query)

# Optionale Anzeige der Anzahl Titel
print(len(records), 'Treffer')

# Optionale Anzeige der Resultate
df

/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

515 Treffer


,MMSID,Titel,035$a,Ort,GND#
0,99116858901405511,"""E Hutte voll Zyt""",(swissbib)235721913-41slsp_network \ 235721913...,Sigriswil§Kanton Bern,(DE-588)4054953-7§(DE-588)4005765-3
1,99116915175905511,"""Weisch no""",(swissbib)044506414-41slsp_network \ 044506414...,Muri bei Bern,(DE-588)4101790-0
2,99116771141905511,[Schwarzenburger Ansichten],(swissbib)038049139-41slsp_network \ 038049139...,Schwarzenburg,(DE-588)4238357-2
3,99116808877605511,100 Jahre Bern Bümpliz,(swissbib)073452122-41slsp_network \ 073452122...,Bern-Bümpliz,(DE-588)4348230-2
4,99116772364105511,100 Jahre Wyssachen,(swissbib)237345404-41slsp_network \ 237345404...,Wyssachen§Kanton Bern,(DE-588)4550339-4§(DE-588)4005765-3
...,...,...,...,...,...
510,99116833347705511,Zollikofen,(swissbib)235925853-41slsp_network \ 235925853...,Zollikofen (Suisse)§Zollikofen,(RERO)A010169409§(DE-588)4268539-4
511,99116795753505511,Zollikofen einst und jetzt,(swissbib)031237436-41slsp_network \ 031237436...,Zollikofen,(DE-588)4268539-4
512,99116877987105511,Zum Beispiel Thun,(swissbib)235387037-41slsp_network \ 235387037...,Thun,(DE-588)4059991-7
513,99116758439305511,Zur Geschichte der Kirchgemeinde Leissigen,(swissbib)033404968-41slsp_network \ 033404968...,Reformierte Kirche Leissigen,(DE-588)7674215-5
